In [19]:
# Data Processing from base data to dataset for Truth-in-Tax vs Statutory rate counterfactual

# Note that personal property has been excluded from this analysis

In [20]:
import pandas as pd

df_clean = pd.read_json("../../public/data/base_data.json")


# drop needless columns
df_clean = df_clean[['Tax Year', 'County Name', 'Entity Name', 'Entity Code', 'County Code', 'Entity Type', 'Tax Rate', 
                 'Total Taxable Property', 'Personal Property', 'Revenue, Total']]

In [21]:
# Compute tax rates without the Truth in Tax constraint

df_clean["Counterfactual Tax Rate"] = df_clean["Tax Rate"]

years = df_clean["Tax Year"].unique().tolist()
entities = df_clean['Entity Name'].unique().tolist()

for year in years:
    for entity in entities:
        df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity), 'Counterfactual Tax Rate'] = max( df_clean.loc[(df_clean["Tax Year"] == year -1 ) & (df_clean['Entity Name']== entity), 'Counterfactual Tax Rate'].sum(), df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity), 'Tax Rate'].sum())
        

In [28]:
# Compute Counterfactual Revenue

df_clean['Base Revenue'] = ((df_clean['Total Taxable Property'] - df_clean['Personal Property'])  * df_clean['Tax Rate']) * 10

df_clean['Counterfactual Revenue'] = ((df_clean['Total Taxable Property'] - df_clean['Personal Property'])  * df_clean['Counterfactual Tax Rate']) *10 

In [33]:
# Compute Truth Dividend

df_clean['Rate Difference'] = df_clean['Counterfactual Tax Rate'] - df_clean['Tax Rate']

df_clean['Truth Dividend'] = df_clean['Counterfactual Revenue'] - df_clean['Base Revenue']

In [34]:
# Export to JSON
df_clean.to_json('../../public/data/counterfactual_data.json', orient = 'records')